In [ ]:
import math
import pickle
import os
import json
from metrics_tools import *
import sklearn.metrics

# set resolution from ["10x", "20x", "40x"]
res = "10x"

# set model cancer threshold
patch_cancer_threshold = 0.5

# set list of subclasses
short_names = ['BrCA', 'Fat', 'Lymphocytes', 'Capsule', 'Blood', 'Histiocytes', 'Vein', 'Artery', 'GC', 'Sinus', 'Mantle', 'Nerve', 'Marginal', 'Light Zone', 'Dark Zone']

# set inference and ground truth directories
inf_dir = None
gt_dir = None 

# set pickle file names
gt_pickle_file = res + "_gt.pickle"
results_pickle_file = res + "_results.pickle"
results_fov_pickle_file = res + "_results_fov.pickle"

In [ ]:
# parse inference and ground truth files into inf and gt
inf, gt = inf_gt_preprocessing(inf_dir, gt_dir)

In [ ]:
# save gt as pickle
with open(gt_pickle_file, 'wb') as file:
    pickle.dump(gt, file)

In [ ]:
# load gt as pickle
with open(gt_pickle_file, 'rb') as file:
    gt = pickle.load(file)

In [ ]:
# compute patch level results
results = {}
for s in short_names:
    results[s] = {'tp':[], 'tn':[], 'fp':[], 'fn':[], 'y_true':[], 'y_pred':[], 'fpr':[], 'tpr':[], 'thresh':[]}

for slide in gt.keys():
    for fov in gt[slide].keys():
        for patch in gt[slide][fov]:
            if patch['object_class'] == 'Benign':
                if patch['conf'][0] >= 1-patch_cancer_threshold:
                    results[patch['short_name']]['tn'].append([slide,fov])
                else:
                    results[patch['short_name']]['fp'].append([slide,fov])
            if patch['object_class'] == 'Cancer':
                if patch['conf'][1] > patch_cancer_threshold:
                    results[patch['short_name']]['tp'].append([slide,fov])
                else:
                    results[patch['short_name']]['fn'].append([slide,fov])
            results[patch['short_name']]['y_true'].append(patch['object_class'])
            results[patch['short_name']]['y_pred'].append(patch['conf'][1])

# get overall results by combining results for each short_name
results['overall'] = {'tp':[], 'tn':[], 'fp':[], 'fn':[], 'y_true':[], 'y_pred':[], 'tpr':[], 'fpr':[], 'thresh':[]}
for key in results['overall']:
    results['overall'][key] = [item for short_name in short_names for item in results[short_name][key]]
    
# get results without fat
nofat = list(short_names)
nofat.remove('Fat')
results['nofat'] = {'tp':[], 'tn':[], 'fp':[], 'fn':[], 'y_true':[], 'y_pred':[], 'tpr':[], 'fpr':[], 'thresh':[]}
for key in results['nofat']:
    results['nofat'][key] = [item for short_name in nofat for item in results[short_name][key]]
    
# remove short_names with no instances
for s in short_names:
    if not results[s]['y_true']:
        results.pop(s)

# get fpr and tpr for various thresholds between 0 and 1
for s in results.keys():
    results[s]['fpr'], results[s]['tpr'], results[s]['thresh'] = sklearn.metrics.roc_curve(results[s]['y_true'], results[s]['y_pred'], drop_intermediate=False, pos_label='Cancer')
    
#get total number and overall accuracy for each short_name
for key in results.keys():
    tp, tn, fp, fn = get_conf_matrix(results[key])
    total = tp + tn + fp + fn
    results[key]['total'] = total
    results[key]['accuracy'] = (tp + tn) / total

In [ ]:
# save results as pickle
with open(results_pickle_file, 'wb') as file:
    pickle.dump(results, file)

In [ ]:
# load results as pickle
with open(results_pickle_file, 'rb') as file:
    results = pickle.load(file)

In [ ]:
# print total number of AOIs and FOVs
print("Total AOIs:")
print(sum(get_conf_matrix(results['overall'])))
print("Total FOVs:")
print(sum([1 for slide in gt.keys() for fov in gt[slide]]))
print("\n")

# print patch level global metrics
print("Overall metrics:")
roc_auc = sklearn.metrics.auc(results['overall']['fpr'], results['overall']['tpr'])
print('AUC:', roc_auc)
print_metrics(results['overall'])
print("\n")

# print patch level global metrics without fat
print("No fat metrics:")
nofat_roc_auc = sklearn.metrics.auc(results['nofat']['fpr'], results['nofat']['tpr'])
print('AUC:', nofat_roc_auc)
print_metrics(results['nofat'])

# print percent and accuracy for each subclass
for key in results.keys():
    print("\n", key)
    print("Accuracy:", round(results[key]['accuracy'], 4))
    print("Percent:", str(round(results[key]['total'] / results['overall']['total'] * 100, 3)) + '%')

In [ ]:
# aggregate_max to FOV level 
fov_cancer_threshold = 0.5
num_patches = 1

gt_fov = {}
for slide in gt.keys():
    gt_fov[slide] = []
    for fov in gt[slide].keys():
        if len(gt[slide][fov]) >= num_patches:
            gt_fov[slide].append(aggregate_max(fov, gt[slide][fov]))

In [ ]:
# compute fov level results
results = {'y_true':[], 'y_pred':[], 'tp':[], 'tn':[], 'fp':[], 'fn':[], 'fpr':[], 'tpr':[], 'thresh':[]}

for slide in gt_fov.keys():
    for fov in gt_fov[slide]:
        if fov['object_class'] == 'Benign':
            if fov['conf'][0] >= 1-fov_cancer_threshold:
                results['tn'].append([slide, fov['id']])
            else:
                results['fp'].append([slide, fov['id']])
        if fov['object_class'] == 'Cancer':
            if fov['conf'][1] > fov_cancer_threshold:
                results['tp'].append([slide, fov['id']])
            else:
                results['fn'].append([slide, fov['id']])
        results['y_true'].append(fov['object_class'])
        results['y_pred'].append(fov['conf'][1])

In [ ]:
# print fov level global metrics
results['fpr'], results['tpr'], results['thresh'] = sklearn.metrics.roc_curve(results['y_true'], results['y_pred'], pos_label='Cancer')
roc_auc = sklearn.metrics.auc(results['fpr'], results['tpr'])
print("AUC:", roc_auc)
print_metrics(results)

In [ ]:
# save results as pickle
with open(results_fov_pickle_file, 'wb') as file:
    pickle.dump(results, file)